In [3]:
import requests
from openpyxl import Workbook

# Función para obtener los datos de una sección
def obtener_datos(seccion):
    url = f"http://localhost:3001/{seccion}/"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error al obtener los {seccion}. Código de estado:", response.status_code)
        return []

# Secciones que deseas obtener de la API
secciones = ["productos", "categorias", "estados", "marcas", "roles", "usuarios", "ventas"]

# Iterar sobre las secciones y guardar los datos en archivos de Excel diferentes
for seccion in secciones:
    datos = obtener_datos(seccion)
    if datos:
        # Crear un nuevo archivo de Excel
        workbook = Workbook()
        sheet = workbook.active

        # Escribir los encabezados en la primera fila del archivo de Excel (excluir 'id' si existe)
        headers = [header for header in datos[0].keys() if not header.lower().startswith('id')]
        for col in range(len(headers)):
            sheet.cell(row=1, column=col + 1, value=headers[col])

        # Iterar sobre los datos y escribir la información en las filas siguientes
        for row, dato in enumerate(datos, start=2):
            for col, header in enumerate(headers):
                sheet.cell(row=row, column=col + 1, value=dato.get(header, 'No data found'))

        # Guardar el archivo de Excel en la computadora local
        workbook.save(f"{seccion}.xlsx")
        print(f"{seccion} guardados en {seccion}.xlsx")



productos guardados en productos.xlsx
categorias guardados en categorias.xlsx
estados guardados en estados.xlsx
marcas guardados en marcas.xlsx
roles guardados en roles.xlsx
usuarios guardados en usuarios.xlsx
ventas guardados en ventas.xlsx


In [4]:
import requests
import pandas as pd
from collections import defaultdict
from decimal import Decimal

# URL de la API local para ventas y productos (ajusta las rutas y puertos según tu configuración)
url_ventas = 'http://localhost:3001/ventas'
url_productos = 'http://localhost:3001/productos'

# Realizar solicitudes a las APIs
response_ventas = requests.get(url_ventas)
response_productos = requests.get(url_productos)

# Verificar el estado de las respuestas
if response_ventas.status_code == 200 and response_productos.status_code == 200:
    # Convertir las respuestas a JSON
    data_ventas = response_ventas.json()
    data_productos = response_productos.json()

    # Crear DataFrames con los datos
    df_ventas = pd.DataFrame(data_ventas)
    df_productos = pd.DataFrame(data_productos)

    # Inicializar una variable para almacenar la suma total de ventas
    suma_total_ventas = Decimal(0)

    # Supongamos que las API devuelven listas de ventas y productos con campos "id_producto", "cantidad" y "precio"
    for venta in data_ventas:
        id_producto = venta.get('id_producto', 0)
        cantidad_venta = int(venta.get('cantidad', 0))
        precio_producto = Decimal([producto['precio'] for producto in data_productos if producto['id_producto'] == id_producto][0])
        total_venta = cantidad_venta * precio_producto

        # Acumular la venta en la suma total
        suma_total_ventas += total_venta

    # Convertir el total de ventas a un número entero
    suma_total_ventas = int(suma_total_ventas)

    # Encontrar la categoría más vendida
    id_categoria_mas_vendida = df_productos['id_producto'].value_counts().idxmax()
    nombre_categoria_mas_vendida = df_productos.loc[df_productos['id_producto'] == id_categoria_mas_vendida, 'nombre_categoria'].values[0]

    # Encontrar el producto más vendido
    productos_vendidos = defaultdict(int)
    for venta in data_ventas:
        producto = venta['nombre_producto']
        cantidad_vendida = venta['cantidad']
        productos_vendidos[producto] += cantidad_vendida

    producto_mas_vendido = max(productos_vendidos, key=productos_vendidos.get)
    cantidad_mas_vendida = productos_vendidos[producto_mas_vendido]

    # Mostrar resultados
    print(f'Ingreso total de todas las ventas: $ {suma_total_ventas} pesos')
    print("Categoría más vendida:", nombre_categoria_mas_vendida)
    print(f'El producto más vendido es "{producto_mas_vendido}" con {cantidad_mas_vendida} unidades vendidas.')

    # Guardar los resultados en un archivo CSV
    resultados = {
        'Ingreso Total': [suma_total_ventas],
        'Categoría Más Vendida': [nombre_categoria_mas_vendida],
        'Producto Más Vendido': [f'{producto_mas_vendido} ({cantidad_mas_vendida} unidades vendidas)']
    }

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_ventas.csv', index=False)
else:
    print("Error al obtener datos de las APIs")

Ingreso total de todas las ventas: $ 22085 pesos
Categoría más vendida: Electrónicos
El producto más vendido es "Muñeca Interactiva DreamDoll" con 8 unidades vendidas.


In [5]:
import requests
import pandas as pd

# URLs de las APIs
url_api_ventas = 'http://localhost:3001/ventas'
url_api_usuarios = 'http://localhost:3001/usuarios'

# Hacer solicitudes a las APIs
response_ventas = requests.get(url_api_ventas)
response_usuarios = requests.get(url_api_usuarios)

# Verificar el estado de las respuestas
if response_ventas.status_code == 200 and response_usuarios.status_code == 200:
    # Convertir las respuestas a JSON
    data_ventas = response_ventas.json()
    data_usuarios = response_usuarios.json()

    # Crear DataFrames con los datos
    df_ventas = pd.DataFrame(data_ventas)
    df_usuarios = pd.DataFrame(data_usuarios)

    # Encontrar el usuario que más ha gastado
    usuario_mas_gastador = df_ventas.groupby('nombre_usuario')['cantidad'].sum().idxmax()

    # Calcular el promedio de gasto por usuario
    promedio_gasto_por_usuario = df_ventas.groupby('id_usuario')['cantidad'].mean().mean()

    # Calcular cuántos productos compra un usuario en promedio
    promedio_productos_por_usuario = df_ventas.groupby('id_usuario')['id_producto'].nunique().mean()

    # Mostrar resultados
    print("Usuario que más ha gastado:", usuario_mas_gastador)
    print("Promedio de gasto por usuario:", promedio_gasto_por_usuario)
    print("Cantidad promedio de productos comprados por usuario:", promedio_productos_por_usuario)

    # Guardar los resultados en un archivo Excel
    resultados = {
        'Usuario Más Gastador': [usuario_mas_gastador],
        'Promedio Gasto por Usuario': [promedio_gasto_por_usuario],
        'Promedio Productos por Usuario': [promedio_productos_por_usuario]
    }

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel('resultados_usuarios.xlsx', index=False)
else:
    print("Error al obtener datos de las APIs")

Usuario que más ha gastado: Usuario1
Promedio de gasto por usuario: 1.9583333333333333
Cantidad promedio de productos comprados por usuario: 4.25


In [6]:
import requests
import pandas as pd

# URLs de las APIs
url_api_productos = 'http://localhost:3001/productos'

# Hacer solicitud a la API de productos
response_productos = requests.get(url_api_productos)

# Verificar el estado de la respuesta
if response_productos.status_code == 200:
    # Convertir la respuesta a JSON
    data_productos = response_productos.json()

    # Crear un DataFrame con los datos de productos
    df_productos = pd.DataFrame(data_productos)

    # Calcular cuántos productos están actualmente en stock
    productos_en_stock = df_productos['stock'].sum()

    # Encontrar el producto con menos stock
    producto_menos_stock = df_productos.loc[df_productos['stock'].idxmin()]['nombre_producto']

    # Calcular cuántos productos necesitan ser reabastecidos (stock bajo)
    productos_stock_bajo = df_productos[df_productos['stock'] < 10]['stock'].count()

    # Mostrar resultados
    print("Cantidad de productos en stock:", productos_en_stock)
    print("Producto con menos stock:", producto_menos_stock)
    print("Cantidad de productos que necesitan ser reabastecidos (stock bajo):", productos_stock_bajo)

    # Guardar los resultados en un archivo Excel
    resultados = {
        'Productos en Stock': [productos_en_stock],
        'Producto Menos Stock': [producto_menos_stock],
        'Productos Stock Bajo': [productos_stock_bajo]
    }

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel('resultados_inventario.xlsx', index=False)
else:
    print("Error al obtener datos de la API de productos")

Cantidad de productos en stock: 1495
Producto con menos stock: Telescopio Terrestre NatureView
Cantidad de productos que necesitan ser reabastecidos (stock bajo): 0


In [11]:
import requests
import pandas as pd
from collections import defaultdict
from decimal import Decimal

# URL de la API local para ventas y productos (ajusta las rutas y puertos según tu configuración)
url_ventas = 'http://localhost:3001/ventas'
url_productos = 'http://localhost:3001/productos'

# Realizar solicitudes a las APIs
response_ventas = requests.get(url_ventas)
response_productos = requests.get(url_productos)

# Verificar el estado de las respuestas
if response_ventas.status_code == 200 and response_productos.status_code == 200:
    # Convertir las respuestas a JSON
    data_ventas = response_ventas.json()
    data_productos = response_productos.json()

    # Crear DataFrames con los datos
    df_ventas = pd.DataFrame(data_ventas)
    df_productos = pd.DataFrame(data_productos)

    # Inicializar una variable para almacenar la suma total de ventas
    suma_total_ventas = Decimal(0)

    # Supongamos que las API devuelven listas de ventas y productos con campos "id_producto", "cantidad" y "precio"
    for venta in data_ventas:
        id_producto = venta.get('id_producto', 0)
        cantidad_venta = int(venta.get('cantidad', 0))
        precio_producto = Decimal([producto['precio'] for producto in data_productos if producto['id_producto'] == id_producto][0])
        total_venta = cantidad_venta * precio_producto

        # Acumular la venta en la suma total
        suma_total_ventas += total_venta

    # Convertir el total de ventas a un número entero
    suma_total_ventas = int(suma_total_ventas)

    # Encontrar la categoría más vendida
    id_categoria_mas_vendida = df_productos['id_producto'].value_counts().idxmax()
    nombre_categoria_mas_vendida = df_productos.loc[df_productos['id_producto'] == id_categoria_mas_vendida, 'nombre_categoria'].values[0]

    # Encontrar el producto más vendido
    productos_vendidos = defaultdict(int)
    for venta in data_ventas:
        producto = venta['nombre_producto']
        cantidad_vendida = venta['cantidad']
        productos_vendidos[producto] += cantidad_vendida

    producto_mas_vendido = max(productos_vendidos, key=productos_vendidos.get)
    cantidad_mas_vendida = productos_vendidos[producto_mas_vendido]

    # Mostrar resultados
    print(f'Ingreso total de todas las ventas: $ {suma_total_ventas} pesos')
    print("Categoría más vendida:", nombre_categoria_mas_vendida)
    print(f'El producto más vendido es "{producto_mas_vendido}" con {cantidad_mas_vendida} unidades vendidas.')

    # Guardar los resultados en un archivo CSV
    resultados = {
        'Ingreso Total': [suma_total_ventas],
        'Categoría Más Vendida': [nombre_categoria_mas_vendida],
        'Producto Más Vendido': [f'{producto_mas_vendido} ({cantidad_mas_vendida} unidades vendidas)']
    }

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_ventas.csv', index=False, encoding='utf-8-sig')  # Cambiado a 'utf-8-sig'
else:
    print("Error al obtener datos de las APIs")


Ingreso total de todas las ventas: $ 22085 pesos
Categoría más vendida: Electrónicos
El producto más vendido es "Muñeca Interactiva DreamDoll" con 8 unidades vendidas.
